# Preparación de datos

En este Notebook se hace realiza una primera revisión de los datos, chequeo de valores faltantes, duplicados y dimensión. Luego se divide el set de datos en **Train** y **Test**.

## Paquetes y Funciones

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [94]:
def data_simple_test(data_dict):
    """Permite imprimir una evaluación rápida de uno o más datasets que modifiquemos.
       Muestra número de filas y de columnas, valores faltantes y duplicados.

    Args:
        data_dict (dict): Diccionario con dataframes a evaluar. Key será el nombre del dataframe.
                          Ejemplo: {'Train':train}
    """
    print("Data Simple Test")
    for data in data_dict:
        print(f'{data} | rows : {data_dict[data].shape[0]} | cols : {data_dict[data].shape[1]} \n        | na : {data_dict[data].isna().sum().sum()} | duplicates : {data_dict[data].duplicated().sum()}')
    
    return None    

## Dataset

In [58]:
data = pd.read_csv("dataset_kickstarter.csv", index_col=0)
data.head()

,fecha_inicial,fecha_final,año_inicial,mes_inicial,dia_inicial,año_final,mes_final,dia_final,pais,ciudad,...,title,cotizacion,moneda_igual_pais,dia_semana_inicial,dia_semana_final,weekend_inicial,weekend_final,primera_quincena_inicial,primera_quincena_final,id
1,2019-04-23,2019-05-23,2019,4,23,2019,5,23,Australia,Brisbane,...,Lands of CROM Trading Card Game,1.000000,0,martes,jueves,0,0,0,0,1
2,2019-03-26,2019-05-24,2019,3,26,2019,5,24,Sweden,Gävle,...,Pastillfabriken - nu bygger vi Gävles nygamla ...,0.108288,1,martes,viernes,0,0,0,0,2
3,2019-04-14,2019-05-05,2019,4,14,2019,5,5,Germany,Munich,...,"Das Brett - cut it, keep it, store it, love it.",1.130450,1,domingo,domingo,1,1,1,1,3
4,2019-04-03,2019-06-02,2019,4,3,2019,6,2,Canada,Montreal,...,Plateforme collaborative.,1.000000,0,miércoles,domingo,0,1,1,1,4
5,2019-03-12,2019-05-11,2019,3,12,2019,5,11,United Kingdom,London,...,Live the story you want to tell,1.307600,1,martes,sábado,0,1,1,1,5


In [59]:
data.shape

(7398, 48)

In [60]:
data.columns

Index(['fecha_inicial', 'fecha_final', 'año_inicial', 'mes_inicial',
       'dia_inicial', 'año_final', 'mes_final', 'dia_final', 'pais', 'ciudad',
       'categoria_principal', 'categoria', 'proyecto_featured',
       'descripcion_largo', 'descripcion_full_largo', 'titulo_largo',
       'cant_creaciones_dueño', 'cant_faqs', 'cant_imagenes', 'cant_videos',
       'cant_tiers', 'tier_promedio', 'tier_max', 'tier_min', 'duracion',
       'moneda', 'objetivo_monto', 'objetivo_ratio', 'monto_obtenido',
       'monto_obtenido_porcentaje', 'estado', 'exito', 'cant_actualizaciones',
       'cant_backers', 'cant_backers_tiers', 'cant_comentarios',
       'backers_promedio_por_tier', 'aporte_promedio_por_backer', 'title',
       'cotizacion', 'moneda_igual_pais', 'dia_semana_inicial',
       'dia_semana_final', 'weekend_inicial', 'weekend_final',
       'primera_quincena_inicial', 'primera_quincena_final', 'id'],
      dtype='object')

No tenemos valores faltantes.

In [61]:
data.isna().sum().sum()

0

Aunque elimine la variable `id` no hay proyectos duplicados en el dataset.

In [62]:
data.drop('id', axis = 1).duplicated().sum()

0

## Train and Test split

Dividimos el dataset en set de Train (75%) y Test (15%). Primero clasificamos las variables en explicativas, futuras y target.

In [63]:
explicativas = ['fecha_inicial', 'fecha_final', 'año_inicial', 'mes_inicial','dia_inicial', 
                'año_final', 'mes_final', 'dia_final', 'pais', 'ciudad', 'descripcion_largo', 
                'descripcion_full_largo', 'titulo_largo','cant_creaciones_dueño', 'cant_faqs', 
                'cant_imagenes', 'cant_videos','cant_tiers', 'tier_promedio', 'tier_max', 
                'tier_min', 'duracion','moneda', 'objetivo_monto', 'objetivo_ratio','title',
                'cotizacion', 'moneda_igual_pais', 'dia_semana_inicial',
                'dia_semana_final', 'weekend_inicial', 'weekend_final',
                'primera_quincena_inicial', 'primera_quincena_final', 'id']
 
target = ['monto_obtenido','monto_obtenido_porcentaje', 'estado', 'exito']
 
futuras = ['cant_actualizaciones', 'cant_backers', 'cant_backers_tiers', 'cant_comentarios',
       'backers_promedio_por_tier', 'aporte_promedio_por_backer']

In [64]:
X = data.drop(target+futuras, axis = 1)
y = data[target+['id']]

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 60, shuffle = True, stratify = y["exito"])

Revisamos los resultados del split de los datos.

In [96]:
split_data_dict = {'Train X':X_train, 
                   'Test X ':X_test, 
                   'Train y':y_train, 
                   'Test y ':y_test}

data_simple_test(split_data_dict)

Data Simple Test
Train X | rows : 6288 | cols : 38 
        | na : 0 | duplicates : 0
Test X  | rows : 1110 | cols : 38 
        | na : 0 | duplicates : 0
Train y | rows : 6288 | cols : 5 
        | na : 0 | duplicates : 0
Test y  | rows : 1110 | cols : 5 
        | na : 0 | duplicates : 0


## Exportamos Splits

In [67]:
train = X_train.merge(y_train, how='inner', on=['id'])
test = X_test.merge(y_test, how='inner', on=['id'])

In [100]:
data_dict = {'Train  ':train, 
             'Test   ':test}

data_simple_test(data_dict)

Data Simple Test
Train   | rows : 6288 | cols : 42 
        | na : 0 | duplicates : 0
Test    | rows : 1110 | cols : 42 
        | na : 0 | duplicates : 0


In [102]:
train.to_csv("Train.csv")
test.to_csv("Test.csv")